In [1]:
import numpy as np
import sys
import scipy.stats
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *

# Functional Example

In [6]:
chrom = FunctionalChromatogram("normal", [0,1])
frag1 = MSN(50,2,0.5)
frag2 = MSN(55,2,0.7)
chem = UnknownChemical(100, 100, 10000, chrom, [frag1, frag2])
frag1.parent = chem
frag2.parent = chem

frag1_1 = MSN(30,3,0.3,None,frag1)
frag1_2 = MSN(35,3,0.5,None,frag1)
frag1.children = [frag1_1, frag1_2]

In [7]:
print(chem.isotopes)
print(chem.adducts)

[(100, 1, 'Mono')]
[('M+H', 1)]


In [8]:
chem.get_all_mz_peaks(101,1,[[(0,300)]])

[(98.992724, 1164.2509265114713)]

In [9]:
chem.get_all_mz_peaks(101,2,[[(0,300)]])

[(48.992724, 582.1254632557357), (53.992724, 814.9756485580299)]

In [10]:
chem.get_all_mz_peaks(101,3,[[(0,300)],[(0,300)]])

[(28.992724, 174.6376389767207), (33.992724, 291.06273162786783)]

# Emprical Example

In [11]:
chrom = FunctionalChromatogram("normal", [0,1])
frag1 = MSN(50,2,0.5)
frag2 = MSN(55,2,0.7)
chrom = FunctionalChromatogram("normal", [0,1])
formula = Formula("C12HgFS",200)
isotopes = Isotopes(formula)
adducts = Adducts(formula)
chem2 = KnownChemical(formula,isotopes,adducts,100, 10000, chrom, [frag1, frag2])
frag1.parent = chem2
frag2.parent = chem2

frag1_1 = MSN(30,3,0.3,None,frag1)
frag1_2 = MSN(35,3,0.5,None,frag1)
frag1.children = [frag1_1, frag1_2]

In [12]:
print(chem2.adducts)
print(chem2.isotopes)

[('M+H', 1.0)]
[(200, 0.8822479928635149, 'Mono'), (198.9966451622, 0.11775200713648519, '1C13')]


In [13]:
chem2.get_all_mz_peaks(101,1,[[(0,300)]])

[(198.992724, 1027.158043104233), (197.9893691622, 137.09288340723828)]

In [14]:
chem2.get_all_mz_peaks(101,2,[[(0,300)]])

[(48.992724, 513.5790215521165), (53.992724, 719.010630172963)]

In [15]:
chem2.get_all_mz_peaks(101,3,[[(0,300)],[(0,300)]])

[(28.992724, 154.07370646563496), (33.992724, 256.78951077605825)]

# Mass Spec test

In [16]:
ps = load_obj('../models/peak_sampler_4_beers.p')
xcms_output = '../models/beer_ms1_peaks.csv.gz'
chromatograms = ChromatogramCreator(xcms_output)

c:\users\vinny\.virtualenvs\simulator-r22bo2kk\lib\site-packages\sklearn\base.py:251: UserWarning:

Trying to unpickle estimator KernelDensity from version 0.20.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.



0


../codes\VMSfunctions\Common.py:49: RuntimeWarning:

divide by zero encountered in double_scalars



5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [18]:
chemicals = ChemicalCreator(ps, chromatograms)

In [19]:
min_ms1_intensity = 2E5
min_rt = 3*60
max_rt = 21*60
dataset = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 2, "Unknown","Empirical",None, False)

In [21]:
scan_parameters = {
    'isolation_windows': [[(0, 1e3)]], # TODO: change to dictionary?
    'ms_level': 1
}    
sampled_chemicals = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 2, "Unknown","Empirical",None, False)
mass_spec = IndependentMassSpectrometer(sampled_chemicals, scan_parameters)

In [31]:
scan_windows = {
    2: [(0, 20), (0, 1e3)],
}
mass_spec._get_all_mz_peaks(dataset[0], 874, 3, [[(0, 1e3)],[(0, 1e3)]])